In [30]:
import pandas as pd
import numpy as np

In [31]:
data = pd.read_csv("data.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2132 entries, 0 to 2131
Data columns (total 43 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Label   2132 non-null   object 
 1   x1      2132 non-null   float64
 2   y1      2132 non-null   float64
 3   x2      2132 non-null   float64
 4   y2      2132 non-null   float64
 5   x3      2132 non-null   float64
 6   y3      2132 non-null   float64
 7   x4      2132 non-null   float64
 8   y4      2132 non-null   float64
 9   x5      2132 non-null   float64
 10  y5      2132 non-null   float64
 11  x6      2132 non-null   float64
 12  y6      2132 non-null   float64
 13  x7      2132 non-null   float64
 14  y7      2132 non-null   float64
 15  x8      2132 non-null   float64
 16  y8      2132 non-null   float64
 17  x9      2132 non-null   float64
 18  y9      2132 non-null   float64
 19  x10     2132 non-null   float64
 20  y10     2132 non-null   float64
 21  x11     2132 non-null   float64
 22  

In [32]:
X = data["Label"]
num_classes = len(X.unique())

X = X.to_numpy()
X.shape

(2132,)

In [33]:
y = data.drop(columns=["Label"])
y = y.to_numpy()
y.shape

(2132, 42)

In [34]:
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [35]:
from tensorflow.keras import layers, Model

def create_model(input_shape=(21*2,), num_classes=num_classes):
    input = layers.Input(shape=input_shape, name="input_layer")

    # Classification Block
    x = layers.Dense(42, activation="relu", name="fc1")(input)
    x = layers.BatchNormalization(name="bn1")(x)
    x = layers.Dropout(0.3, name="dropout_1")(x)
    x = layers.Dense(36, activation="relu", name="fc2")(x)
    x = layers.BatchNormalization(name="bn2")(x)
    x = layers.Dropout(0.4, name="dropout_2")(x)
    x = layers.Dense(28, activation="relu", name="fc3")(x)
    x = layers.BatchNormalization(name="bn3")(x)
    x = layers.Dropout(0.4, name="dropout_3")(x)

    output = layers.Dense(num_classes, activation="softmax", name="Prediction")(x)
    
    model = Model(inputs=input, outputs = output, name="mnist")

    return model

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from scipy.sparse import issparse

def train_model(model, X, Y, X_val, Y_val):
    with tf.device("/gpu:0"):
        if issparse(X):
            X = X.toarray()
        if issparse(Y):
            Y = Y.toarray()
        if issparse(X_val):
            X_val = X_val.toarray()
        if issparse(Y_val):
            Y_val = Y_val.toarray()

        X = np.asarray(X)
        Y = np.asarray(Y)
        X_val = np.asarray(X_val)
        Y_val = np.asarray(Y_val)

        train_datagen = ImageDataGenerator(
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.3,
            zoom_range=0.3,
            fill_mode='nearest',
            brightness_range=[0.8, 1.2],
            preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
        )

        val_datagen = ImageDataGenerator(
            preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
        )

        model.compile(
            loss='sparse_categorical_crossentropy',
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            metrics=['accuracy']
        )

        batch_size = 1024
        val_size = 1024

        callbacks = [
            tf.keras.callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.2,
                patience=3,
                min_delta=0.001,
                min_lr=1e-10
            ),
            tf.keras.callbacks.ModelCheckpoint(
                './models/best_MNIST_Classifier.keras',
                save_best_only=True,
                monitor='val_accuracy'
            ),
            tf.keras.callbacks.EarlyStopping(
                monitor='val_loss',
                patience=10,
                min_delta=0.001,
                mode='min'
            )
        ]

        train_gen = train_datagen.flow(X, Y, batch_size=batch_size, shuffle=True)
        val_gen = val_datagen.flow(X_val, Y_val, batch_size=val_size, shuffle=True)

        history = model.fit(
            train_gen,
            validation_data=val_gen,
            steps_per_epoch=len(X) // batch_size,
            validation_steps=None,
            epochs=100,
            callbacks=callbacks,
        )

        return history

In [36]:
model = create_model()
model.summary()

Model: "mnist"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 42)             │         1,806 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn1 (BatchNormalization)        │ (None, 42)             │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 36)             │         1,548 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn2 (BatchNormalization)        │ (None, 36)             │           144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 36)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc3 (Dense)                     │ (None, 28)             │         1,036 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bn3 (BatchNormalization)        │ (None, 28)             │           112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 28)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Prediction (Dense)              │ (None, 28)             │           812 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,626 (21.98 KB)

 Trainable params: 5,414 (21.15 KB)

 Non-trainable params: 212 (848.00 B)